# Upward inference notebook

## The basics

### Model structure
Logical Neural Network correspond to a set of logical formulae which can be represented with formulae's syntax trees.

We will start with simple example where LNN can be represented by one [*logical formula*](https://en.wikipedia.org/wiki/First-order_logic#Formulas) with [*logical implication*](https://en.wikipedia.org/wiki/Truth_table#Logical_implication). This can be visualised as following syntax tree.

![implication_graph](images/implication_graph_plain.png)

### Upward inference principle
Upward inference is evaluation of the *operator* given all the *operands*. In our case logical implication is operator and values $a$ and $b$ are operands. Following image illustrate why we use word "upward", upward inference starts on the bottom of the syntax tree and continue upward by evaluating the operands.

![implication_graph](images/implication_graph.png)

In this case we will be evaluating logical implication which is described with following *truth table*. When values $a$ and $b$ are evalueted to $False$ and $True$ respectively, the whole logical implication is evaluated to $False$, which is visible on third row of the table.

![implication_tt](images/implication_tt.png)

### Predicates

In the [First-order logic](https://en.wikipedia.org/wiki/First-order_logic), values $a$ and $b$ follows from specific domain which includes objects and its relations. The relations are called [*predicates*](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29) and they are usually named by capital letters (for example $P$, $Q$, latter we will use names from natural language so do not be confused). The notatiton $P(x)$ describes the value of predicate $P$ for variable $x$ (where $x$ can be some object from our domain).

Predicates can be described by following table (description for predicates $P$ and $Q$). First row tell us that $P(x)$ for $x = object1$ equals $True$ etc.

![predicate_table](images/predicate_table.png)

### Upward inferece with predicates

Lets have formula $P(x) \implies Q(x)$, syntax tree can be visualised in simillar fashion as the previous one.

![implication_graph_predicate](images/implication_graph_predicate.png)

So if we want to evaluate formula $P(x) \implies Q(x)$, we first have to evaluate operands $P(x)$ and $Q(x)$. Lets say that $P(x)$, $Q(x)$ will be evaluated to $True$, $False$ respectively. Next we evaluate implication operand with given operands which gives us $False$ as result of evaluation.

![upward_inference](images/upward_inference.png)

The results of upward inference can be shown in following table where first row describes our objects (also called *constants*), in the two middle columns truth values for both predicates given specific constant are displayed and the last column contains truth value of evaluated formula with given predicates and given constant.

![predicate_tt](images/predicate_tt.png)

## Grecian domain

Now we are ready to move to our specific example and finally create a LNN model. We usually start with formulation of our problem in natural language, that will define our *domain*.

* Grecians are mortal.<br>
* Zeus is a grecian, is not mortal.<br>
* Socrates is a grecian, is mortal.<br>
* Confucius is not a grecian, is mortal.<br>
* Great Pyramid of Giza is not grecian, is not mortal.

### First order logic description
Now let us translate this domain into First-order logic.

#### Predicates
The word "grecian" and "mortal" defines relations to our objects (for example "Socrates is grecian", "Confucius is mortal" etc.), we can create two predicates `grecian(x)` and `mortal(x)`.

#### Formulae
The "Grecians are mortal" sentence can be translated as formula with implication and two predicates.

$grecian(x) \implies mortal(x)$

#### Objects
Our natural language description mentions following set of objects `{Zeus, Socrates, Confucius, Great Pyramid of Giza}`. Their relations are represented by following table:

![grecian_tt](images/grecian_tt_predicates.png)

Syntax tree is shown on following example (note its structurally same as the one above).

![grecian_graph](images/grecian_graph.png)

### Our goal
We would like to infer truth values for formula which represents sentence "Grecians are mortal" for each object from `{Zeus, Socrates, Confucius, Great Pyramid of Giza}`. Since we do not have truth values for it from our domain description.

![grecian_tt](images/grecian_tt.png)

### Definition of LNN model

Now we will define our model in LNN library.

First we will create an empty model.

In [1]:
from lnn import Model

model = Model()

Define our predicates and add them to the model. Note we add predicates as formulae, since predicates are in fact atomic formulae.

In [2]:
from lnn import Predicate

grecian = Predicate(name="grecian")
mortal = Predicate(name="mortal")

model.add_formulae(grecian, mortal)

Now we create one variable and use it in definition of our implication `grecians_are_mortals`, which is later added into our model.

In [3]:
from lnn import Variable, Implies

x = Variable("x")

grecians_are_mortals = Implies(grecian(x), mortal(x), name="grecians_are_mortals")

model.add_formulae(grecians_are_mortals)

Last thing is adding so called *facts* into our model. Facts are objects which were mentioned above.

In [4]:
from lnn import Fact

model.add_facts(
    {
        "grecian": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.FALSE,
            "Zeus": Fact.TRUE,
            "Socrates": Fact.TRUE,
        },
        "mortal": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.TRUE,
            "Zeus": Fact.FALSE,
            "Socrates": Fact.TRUE,
        },
    }
)


### Upward inference inside LNN model

Now we have everything ready for upward inference. The upward inference starts from atoms (two predicates in our case) and compute truth bounds for each subformula (implication in our case).

In [5]:
from lnn import Direction

steps, facts_inferred = model.infer(direction=Direction.UPWARD)

Expected result of upward inference in our domain can be seen in following table.

![grecian_tt](images/grecian_tt_infered.png)

We can easily verify truth values for `grecians_are_mortals`.

In [6]:
model["grecians_are_mortals"].print()

OPEN   Implies: grecians_are_mortals(x) 
'Confucius'                                                 TRUE (1.0, 1.0)
'Great Pyramid of Giza'                                     TRUE (1.0, 1.0)
'Socrates'                                                  TRUE (1.0, 1.0)
'Zeus'                                                     FALSE (0.0, 0.0)

